# Lending Club Loan Data Clustering & Seed Instruction Generation

## 📦 Environment Setup
- Installs necessary libraries for data analysis


In [1]:
!pip install -q hvplot

## 📊 Data Loading & Preparation
- Loads dataset from Lending Club loan data CSV
- Performs train/test split (66%/34%) with random sampling
- Handles data persistence through pickling for reproducibility


In [2]:
import hvplot.pandas
import pandas as pd
import numpy as np
SaveSplits=False
df = pd.read_csv("lending_club_loan_two.csv")
data=df.dropna()
import random
import pickle
#train, test,  = train_test_split(data2, test_size=0.33, random_state=42)
random.seed(42)
Ind=list(range(len(data)))
random.shuffle(Ind)
trainInd=Ind[:int(0.66*len(data))]
testInd=Ind[int(0.66*len(data)):]
OrigData=data.copy()
data=OrigData.iloc[trainInd]
test=OrigData.iloc[testInd]

filename = 'Indices.pkl'
if SaveSplits==False:
with open(filename, 'wb') as file:
    pickle.dump(trainInd, file)
    pickle.dump(testInd, file)
    pickle.dump(data, file)
    pickle.dump(test, file)


#Load the objects from the file
with open(filename, 'rb') as file:
    trainInd = pickle.load(file)
    testInd = pickle.load(file)
    data = pickle.load(file)
    test = pickle.load(file)



data['loan_status'] = data.loan_status.map({'Fully Paid':1, 'Charged Off':0})
OrigDFKeys=list(data.keys())
OrigDFKeys.append('zip_code')
pd.set_option('display.max_columns',100)
data

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
232372,24000.0,36 months,15.99,843.66,D,D2,Account Executive,< 1 year,MORTGAGE,95000.0,Verified,Nov-2014,1,home_improvement,Home improvement,14.31,Oct-1994,16.0,0.0,13468.0,64.1,41.0,f,INDIVIDUAL,2.0,0.0,"682 Aimee Vista Apt. 857\r\nEast Paulfurt, AK ..."
226851,7125.0,36 months,16.29,251.52,C,C4,"BETHANY HOME CARE,INC.",5 years,RENT,28000.0,Source Verified,Aug-2012,0,other,Personal loan,11.50,Nov-2006,8.0,0.0,4737.0,66.3,12.0,f,INDIVIDUAL,0.0,0.0,"585 Soto Row Apt. 014\r\nWest Williamberg, TN ..."
35482,9500.0,36 months,14.33,326.22,C,C1,Novare Group,7 years,MORTGAGE,65000.0,Source Verified,Dec-2012,1,credit_card,Credit Card Consolidation,14.82,Dec-1993,14.0,0.0,10835.0,62.3,33.0,f,INDIVIDUAL,1.0,0.0,"416 Robin Mountain\r\nMichaelside, WV 00813"
298295,20000.0,36 months,14.09,684.43,B,B5,Dr.Marcus Black DDS,< 1 year,OWN,90000.0,Not Verified,Sep-2012,1,debt_consolidation,Debt consolidation,24.75,Dec-1998,20.0,0.0,19509.0,56.1,44.0,f,INDIVIDUAL,2.0,0.0,"157 Jacob Loaf\r\nLeeland, WV 05113"
162626,25450.0,60 months,23.28,721.55,F,F2,round top state bank,1 year,MORTGAGE,75000.0,Verified,Dec-2012,1,debt_consolidation,Debt consolidation,8.13,Mar-1987,11.0,0.0,4528.0,22.8,25.0,f,INDIVIDUAL,0.0,0.0,"9980 Irwin Valleys Suite 322\r\nNew Monicaton,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62899,15000.0,36 months,18.75,547.95,D,D3,Borgata,9 years,RENT,60000.0,Verified,May-2013,0,debt_consolidation,Debt consolidation,28.06,Jan-1995,14.0,0.0,11760.0,43.7,32.0,f,INDIVIDUAL,0.0,0.0,"707 Robert Heights Apt. 660\r\nEmilyfurt, CA 7..."
243673,5200.0,36 months,13.35,176.09,C,C2,Manager - Partner Success,1 year,RENT,110000.0,Verified,Jun-2014,1,debt_consolidation,Debt consolidation,8.38,Feb-1991,7.0,0.0,7865.0,72.2,37.0,w,INDIVIDUAL,1.0,0.0,"092 Jones Trafficway\r\nJudithborough, MT 48052"
145956,15000.0,60 months,12.99,341.22,C,C2,General Manager,6 years,MORTGAGE,46000.0,Source Verified,Dec-2014,1,debt_consolidation,Debt consolidation,21.39,May-2007,6.0,0.0,9157.0,52.3,7.0,w,INDIVIDUAL,1.0,0.0,USNS Lopez\r\nFPO AP 05113
66879,8000.0,36 months,10.99,261.88,B,B3,Respiratory therapist (CRT),10+ years,OWN,42000.0,Verified,Jul-2014,1,credit_card,Credit card refinancing,18.03,Aug-1996,15.0,0.0,6431.0,45.3,23.0,w,INDIVIDUAL,0.0,0.0,Unit 8922 Box 1072\r\nDPO AE 29597


## Data Cleaning & Feature Engineering

- **Feature Removal**: Eliminates redundant/low-value features:
  - `emp_title` (too many unique job titles)
  - `emp_length` (employment duration)
  - `title` (duplicate of purpose)
- **Categorical Handling**:
  - Converts `term` from text to numeric (36/60 months)
  - Creates dummy variables for categorical features
  - Extracts zip code from address field for geospatial analysis
- **Date Processing**:
  - Converts `issue_d` and `earliest_cr_line` to year-based numeric features


In [34]:
# The length of the data
print(f"The Length of the data: {data.shape}")

The Length of the data: (221672, 96)


In [35]:
# Missing values
for column in data.columns:
    if data[column].isna().sum() != 0:
        missing = data[column].isna().sum()
        portion = (missing / data.shape[0]) * 100
        print(f"'{column}': number of missing values '{missing}' ==> '{portion:.3f}%'")

Drops employemnt emp_title, emp_length, and title due to lots of unique values

In [6]:
data.drop('emp_title', axis=1, inplace=True)

In [7]:
data.drop('emp_length', axis=1, inplace=True)

In [8]:
data.drop('title', axis=1, inplace=True)

Drops all rows with na values

In [9]:
data.dropna(inplace=True)

In [10]:
term_values = {' 36 months': 36, ' 60 months': 60}
data['term'] = data.term.map(term_values)

In [12]:
dummies = ['grade','sub_grade', 'verification_status', 'purpose', 'initial_list_status', 
           'application_type', 'home_ownership']
data = pd.get_dummies(data, columns=dummies, drop_first=False)


In [14]:
data['zip_code'] = data.address.apply(lambda x: x[-5:])
ZipToStateDF=data.copy()
ZipToStateDF['zip_code'] = ZipToStateDF.address.apply(lambda x: x[-5:])
ZipToStateDF['state'] = ZipToStateDF.address.apply(lambda x: x[-8:-5])

In [15]:
ZipToState={zipcode:state for (zipcode,state) in ZipToStateDF[['zip_code','state']].values}
ZipToState

{'70466': 'CA ',
 '22690': 'HI ',
 '00813': 'VT ',
 '05113': 'AP ',
 '29597': 'AE ',
 '86630': 'ME ',
 '11650': 'CA ',
 '93700': 'UT ',
 '30723': 'OK ',
 '48052': 'MT '}

In [16]:
data.zip_code.value_counts()

zip_code
70466    31825
30723    31637
22690    31478
48052    31286
00813    25748
05113    25431
29597    25394
93700     6336
11650     6316
86630     6221
Name: count, dtype: int64

In [17]:
data = pd.get_dummies(data, columns=['zip_code'], drop_first=False)

In [18]:
data.drop('address', axis=1, inplace=True)

In [19]:
data['issue_d'] = pd.to_datetime(data['issue_d'])
data['issue_d'] = data.issue_d.dt.year

/tmp/ipykernel_493/2345724880.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['issue_d'] = pd.to_datetime(data['issue_d'])


In [20]:
data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])
data['earliest_cr_line'] = data.earliest_cr_line.dt.year

/tmp/ipykernel_493/1915473461.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])


# 🤖 Clustering & Seed Generation
## Cluster-based Distribution Analysis

- **Clustering Strategy**:
  - Uses KMeans (2 clusters) with standardized features
  - Focuses on key financial metrics (loan amount, interest rate, income, etc.)
- **Seed Instruction Creation**:
  - Generates statistical profiles for each cluster
  - Includes:
    - Continuous variable distributions (mean, std, skew/kurtosis)
    - Categorical distributions (proportions)
    - Key business rules (grade, zip code, loan status)
  - Applies sampling factor to control output size
 



In [24]:
#with grade
data2=data.copy()
del data2['home_ownership_OTHER']
del data2['home_ownership_OWN']
del data2['home_ownership_RENT']

del data2['application_type_INDIVIDUAL']
del data2['application_type_JOINT']

del data2['purpose_credit_card']
del data2['purpose_debt_consolidation']
del data2['purpose_educational']
del data2['purpose_home_improvement']
del data2['purpose_house']
del data2['purpose_major_purchase']
del data2['purpose_medical']
del data2['purpose_moving']
del data2['purpose_other']
del data2['purpose_renewable_energy']
del data2['purpose_small_business']
del data2['purpose_vacation']
del data2['purpose_wedding']



del data2['sub_grade_A1']
del data2['sub_grade_A2']
del data2['sub_grade_B1']
del data2['sub_grade_C1']
del data2['sub_grade_D1']
del data2['sub_grade_E1']
del data2['sub_grade_F1']
del data2['sub_grade_G1']
del data2['sub_grade_A3']
del data2['sub_grade_B2']
del data2['sub_grade_C2']
del data2['sub_grade_D2']
del data2['sub_grade_E2']
del data2['sub_grade_F2']
del data2['sub_grade_G2']
del data2['sub_grade_A4']
del data2['sub_grade_B3']
del data2['sub_grade_C3']
del data2['sub_grade_D3']
del data2['sub_grade_E3']
del data2['sub_grade_F3']
del data2['sub_grade_G3']
del data2['sub_grade_A5']
del data2['sub_grade_B4']
del data2['sub_grade_C4']
del data2['sub_grade_D4']
del data2['sub_grade_E4']
del data2['sub_grade_F4']
del data2['sub_grade_G4']
del data2['sub_grade_B5']
del data2['sub_grade_C5']
del data2['sub_grade_D5']
del data2['sub_grade_E5']
del data2['sub_grade_F5']
del data2['sub_grade_G5']

del data2['verification_status_Source Verified']
del data2['verification_status_Verified']

del data2['open_acc']
del data2['pub_rec']
del data2['revol_util']
del data2['total_acc']
del data2['initial_list_status_w']
del data2['installment']
del data2['revol_bal']


In [25]:
Keys=['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'zip_code_05113',
 'zip_code_11650',
 'zip_code_22690',
 'zip_code_29597',
 'zip_code_30723',
 'zip_code_48052',
 'zip_code_70466',
 'zip_code_86630',
 'zip_code_93700',
 'loan_status']


In [26]:
from sklearn.cluster import KMeans,DBSCAN
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scalerAll=StandardScaler()
#X_ScaledAll=scaler.fit_transform(data)

ClusteringKeys=[
   'loan_amnt',
    'term',
    'int_rate',
    'annual_inc',
    'dti',
    'earliest_cr_line',
    'mort_acc',
    'pub_rec_bankruptcies'
]


In [27]:

ContinuousVariables=[
  'loan_amnt', 
  'int_rate',
  'installment',
  'annual_inc',
  'issue_d',
  'dti',
  'earliest_cr_line',
  'open_acc', 
  'pub_rec',
  'revol_bal',
  'revol_util',
  'total_acc',
  'mort_acc',
  'pub_rec_bankruptcies'
]

PrimaryDiscreteVariables=[
  'grade',
  'zip_code',
]
PrimaryBinaryDiscreteVariables=[
  'loan_status'
]

PrimaryBinaryDiscreteDistributionVariables=[
  'term'
]

      
SecondaryDistributionVariables=[
  'sub_grade',
  'verification_status',
  'purpose',
  'initial_list_status',       
  'application_type',
  'home_ownership',
]

DummyVariables={
  'grade': ['A', 'B', 'C', 'D','E', 'F', 'G'],
  'sub_grade':['1', '2', '3', '4', '5'],
  'verification_status': ['Not Verified', 'Source Verified', 'Verified'],
  'purpose':['car', 'credit_card', 'debt_consolidation', 'educational', 'home_improvement', 'house', 'major_purchase', 'medical', 'moving', 'other','renewable_energy','small_business','vacation', 'wedding'], 
  'initial_list_status': ['f', 'w']  ,       
  'application_type': ['DIRECT_PAY', 'INDIVIDUAL', 'JOINT'],
  'home_ownership': ['RENT','MORTGAGE', 'OWN','ANY', 'NONE', 'OTHER'],
  'zip_code':['00813', '05113', '11650', '22690','29597', '30723', '48052', '70466', '86630', '93700'],
  'loan_status':['Charged Off', 'Paid Fully'],
  'term':[36, 60]
}




## 🧠 Key Design Decisions
1. **Feature Selection**: Focuses on 8 core clustering variables that best represent loan risk profiles
2. **Cluster Replication**: Uses sampling factor to balance representativeness and output size
3. **Statistical Depth**: Captures both central tendencies and distribution shape (skew/kurtosis)
4. **Business Context**: Preserves key categorical relationships (grade-sub_grade, zip_code-state)


In [28]:
GroupsIter=iter(data.groupby(Keys))
print(len(data))
import numpy as np
lens=[]
lens_cluster=[]
FinalSeeds=[]
CurrentSeed=[]
NumOfClusters=2
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
SamplingFactor=100
AllDFLen=[]
for name,df2 in GroupsIter:
    X_Scaled=scaler.fit_transform(df2[ClusteringKeys])
    X_ScaledAll=scaler.fit_transform(df2)

    kmeans=KMeans(n_clusters=NumOfClusters,n_init='auto',random_state=42)
    kmeans.fit(X_ScaledAll)
    
    labels=kmeans.labels_

    for ClusterInd in range(0,NumOfClusters):
        CurrentSeed=[]

        Ind=np.where(kmeans.labels_==ClusterInd)[0]
        ClusterRep=len(Ind)
        df=df2.iloc[Ind]
        CurrGrade='A'
        for key in OrigDFKeys:
            if key in ContinuousVariables:
                m=round(df[key].mean(numeric_only=True),2)
                s=round(df[key].std(numeric_only=True),2)
                kur=round(df[key].kurtosis(numeric_only=True),2)
                skewn=round(df[key].skew(numeric_only=True),2)
                mystr=f'{key}: mean={m}, std={s}, kurtosis={kur}, skewness={skewn}'
                #print(mystr)
            elif key in PrimaryDiscreteVariables:
                for SubKey in DummyVariables[key]:
                  if df[key+'_'+SubKey].all():
                      mystr=key+'='+SubKey
                      if key=='grade':
                        CurrGrade=SubKey
                      if key=='zip_code':
                        mystr='address: ' + 'State zipcode'+'='+ZipToState[SubKey]+SubKey


            elif key in PrimaryBinaryDiscreteVariables:
                if df[key].all() and key=="loan_status":
                    mystr=key+'=Fully Paid'
                elif key=="loan_status":
                    mystr=key+'=Charged Off'

                #print(mystr)
            elif key in SecondaryDistributionVariables:
                SecondaryValues=[]
                SecondaryKeys=[]
                for Skey in DummyVariables[key]:
                  if key!='sub_grade':
                    SecondaryValues.append(df[key+'_'+Skey].sum())
                    SecondaryKeys.append(Skey)
                  else:
                    SecondaryValues.append(df[key+'_'+CurrGrade+Skey].sum())
                    SecondaryKeys.append(CurrGrade+Skey)

                      
                mystr=key + ": "+", ".join([k+'='+str(round(val/np.sum(SecondaryValues),2))  for val,k in zip(SecondaryValues,SecondaryKeys)])
            elif key in PrimaryBinaryDiscreteDistributionVariables:
                SecondaryValues=[]
                SecondaryKeys=[]

                for Skey in DummyVariables[key]:
                    SecondaryValues.append(np.sum(df[key]==Skey))
                    SecondaryKeys.append(Skey)
                if key=="term":
                  mystr=key + ": "+", ".join([str(k)+ " months"+'='+str(round(val/np.sum(SecondaryValues),2))  for val,k in zip(SecondaryValues,SecondaryKeys)])
                else:
                  mystr=key + ": "+", ".join([str(k)+'='+str(round(val/np.sum(SecondaryValues),2))  for val,k in zip(SecondaryValues,SecondaryKeys)])
                #print(s)
            else:
                continue

            CurrentSeed.append(mystr)
        
        AllDFLen.append(ClusterRep)
        for RepCount in range(0,int(ClusterRep/SamplingFactor)):
            FinalSeeds.append(CurrentSeed)

    lens.append(len(df))


221672


In [31]:
random.shuffle(FinalSeeds)
FinalSeeds[0:5]

[['loan_amnt: mean=18272.17, std=7814.85, kurtosis=-0.45, skewness=0.38',
  'term: 36 months=0.81, 60 months=0.19',
  'int_rate: mean=11.09, std=1.45, kurtosis=-0.66, skewness=-0.08',
  'installment: mean=557.01, std=250.39, kurtosis=-0.16, skewness=0.61',
  'grade=B',
  'sub_grade: B1=0.18, B2=0.21, B3=0.22, B4=0.22, B5=0.16',
  'home_ownership: RENT=0.11, MORTGAGE=0.83, OWN=0.06, ANY=0.0, NONE=0.0, OTHER=0.0',
  'annual_inc: mean=100035.54, std=57485.2, kurtosis=33.28, skewness=4.04',
  'verification_status: Not Verified=0.24, Source Verified=0.36, Verified=0.41',
  'issue_d: mean=2013.9, std=1.1, kurtosis=-0.84, skewness=0.17',
  'loan_status=Fully Paid',
  'purpose: car=0.0, credit_card=0.28, debt_consolidation=0.6, educational=0.0, home_improvement=0.08, house=0.0, major_purchase=0.01, medical=0.0, moving=0.0, other=0.01, renewable_energy=0.0, small_business=0.01, vacation=0.0, wedding=0.0',
  'dti: mean=17.48, std=7.64, kurtosis=-0.48, skewness=0.24',
  'earliest_cr_line: mean=19

## 📤 Output Generation

- Exports seed instructions to JSON file
- Format contains 100+ statistical descriptors per cluster
- Ready for use with Synthetic Data Generation (SDS) systems



In [30]:
import json
file_path = "SeedsInstructions.json"

with open(file_path, 'w') as json_file:
    json.dump(FinalSeeds, json_file, indent=4)
